### Tokenization

In [1]:
from nltk.tokenize import word_tokenize
text = "I love programming"
word_tokenize(text)

['I', 'love', 'programming']

In [2]:
text = "I love programming and programming also loves me"
tokens=word_tokenize(text)

In [3]:
tokens

['I', 'love', 'programming', 'and', 'programming', 'also', 'loves', 'me']

### Lemmatization

In [4]:
from nltk.tokenize import word_tokenize
text = "I love programming and programming also loves me"
tokens=word_tokenize(text)

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
tokens=[lemmatizer.lemmatize(word) for word in tokens]
print(tokens)

['I', 'love', 'programming', 'and', 'programming', 'also', 'love', 'me']


### Stemming

In [5]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
text = "I love programming and programming also loves me"
tokens=word_tokenize(text.lower())

ps = PorterStemmer()
tokens=[ps.stem(word) for word in tokens]
print(tokens)

['i', 'love', 'program', 'and', 'program', 'also', 'love', 'me']


### Stop words

In [6]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [7]:
newtokens=[]
for token in tokens:
    if token not in stopwords:
        newtokens.append(token)
        
print(newtokens)

['love', 'program', 'program', 'also', 'love']


### Normalization

In [8]:
tweet = 'China has new "AI Development Plan." I think this will really help Chinese and global AI. http://www.gov.cn/zhengce/content/2017-07/20/content_5211996.htm'
print(tweet)

China has new "AI Development Plan." I think this will really help Chinese and global AI. http://www.gov.cn/zhengce/content/2017-07/20/content_5211996.htm


In [9]:
import re
def normalize_text(text):
    text=text.lower()
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(pic\.twitter\.com/[^\s]+))','', text)
    text = re.sub('@[^\s]+','', text)
    text = re.sub('#([^\s]+)', '', text)
    text = re.sub('[:;>?<=*+()&,\-#!$%\{˜|\}\[^_\\@\]1234567890’‘]',' ', text)
    text = re.sub('[\d]','', text)
    text = text.replace(".", '')
    text = text.replace("'", '')
    text = text.replace("`", '')
    text = text.replace("'s", '')
    text = text.replace("/", ' ')
    text = text.replace("\"", ' ')
    text = text.replace("\\", '')
    #text =  re.sub(r"\b[a-z]\b", "", text)
    text=re.sub( '\s+', ' ', text).strip()
    
    return text

In [10]:
normalized_tweet=normalize_text(tweet)
print(normalized_tweet)

china has new ai development plan i think this will really help chinese and global ai


In [11]:
from nltk.tokenize import word_tokenize
tokens=word_tokenize(normalized_tweet)

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
tokens=[lemmatizer.lemmatize(word) for word in tokens]

newtokens=[]
for token in tokens:
    if token not in stopwords:
        newtokens.append(token)
        
print(set(newtokens))

{'really', 'ai', 'plan', 'think', 'new', 'ha', 'chinese', 'global', 'development', 'china', 'help'}


### Document Vectorization

In [12]:
documents= [' I love programming' , 'Programming also loves me']

In [13]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
docs=[]
for document in documents:
    text=normalize_text(document)
    nl_text=''
    for word in word_tokenize(text):
        if word not in stopwords:
            nl_text+=(lemmatizer.lemmatize(word))+' '
    docs.append(nl_text)    

In [14]:
docs

['love programming ', 'programming also love ']

In [15]:
import collections
words=" ".join(docs).split() 
count= collections.Counter(words).most_common()
print(count)

[('love', 2), ('programming', 2), ('also', 1)]


In [16]:
#create a lexicon [features]
features=[c[0] for c in count]
print(features)

['love', 'programming', 'also']


In [17]:
import numpy as np
training_examples=[]
for doc in docs:
    doc_feature_values = np.zeros(len(features))
    for word in word_tokenize(doc):
        if word in features:
            index=features.index(word)
            doc_feature_values[index] +=1
            
    training_examples.append(doc_feature_values)

In [18]:
training_examples

[array([ 1.,  1.,  0.]), array([ 1.,  1.,  1.])]

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

bow = vectorizer.fit_transform(docs)
freqs = [(word, bow.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]
results=sorted (freqs, key = lambda x: -x[1])
print(results)

[('love', 2), ('programming', 2), ('also', 1)]


In [20]:
import pandas as pd

feature_names = vectorizer.get_feature_names()
corpus_index = [n for n in docs]

df = pd.DataFrame(bow.todense(), index=corpus_index, columns=feature_names)
df

,also,love,programming
love programming,0,1,1
programming also love,1,1,1


### Term frequency and inverse term frequency (TF-IDF)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer1 = TfidfVectorizer(sublinear_tf=True, max_df=1.0)
bow1 = vectorizer1.fit_transform(docs)

freqs1 = [(word, bow1.getcol(idx).sum()) for word, idx in vectorizer1.vocabulary_.items()]
results1=sorted (freqs1, key = lambda x: -x[1])
print(results1)

[('love', 1.2086556882809263), ('programming', 1.2086556882809263), ('also', 0.70490948893093264)]


In [22]:
feature_names = vectorizer1.get_feature_names()
corpus_index = [n for n in docs]

import pandas as pd
df = pd.DataFrame(bow1.todense(), index=corpus_index, columns=feature_names)
df

,also,love,programming
love programming,0.000000,0.707107,0.707107
programming also love,0.704909,0.501549,0.501549
